In [32]:
import pandas as pd
import numpy as np

In [33]:
data = pd.read_csv("./data/malware-dataset.csv")
data

,sha256,labels,0,1,2,3,4,5,6,7,...,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176
0,5c18291c481a192ed5003084dab2d8a117fd3736359218...,0,LdrUnloadDll,CoUninitialize,NtQueryKey,NtDuplicateObject,GetShortPathNameW,GetSystemInfo,IsDebuggerPresent,GetSystemWindowsDirectoryW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4683faf3da550ffb594cf5513c4cbb34f64df85f27fd1c...,0,NtOpenMutant,GetForegroundWindow,NtQueryKey,DrawTextExW,NtSetInformationFile,RegQueryValueExA,LdrGetProcedureAddress,CoUninitialize,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9a0aea1c7290031d7c3429d0e921f107282cc6eab854ee...,0,GetForegroundWindow,DrawTextExW,GetSystemInfo,IsDebuggerPresent,GetSystemWindowsDirectoryW,NtQueryValueKey,RegCloseKey,GetFileAttributesW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e0f3e4d5f50afd9c31e51dd9941c5a52d57c7c524f5d11...,0,NtQueryValueKey,LdrUnloadDll,GlobalMemoryStatus,WriteConsoleA,NtOpenKey,LdrGetProcedureAddress,NtTerminateProcess,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ec2b6d29992f13e74015ff0b129150b4afae15c593e4b7...,0,LdrUnloadDll,GetSystemTimeAsFileTime,NtOpenKey,WSAStartup,SetUnhandledExceptionFilter,NtTerminateProcess,NtClose,NtAllocateVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565,ed6a3fc04af435176b9c2f3024eb53c31d1e522da52c5c...,1,CreateToolhelp32Snapshot,GetCursorPos,CoUninitialize,RegCloseKey,LdrUnloadDll,DrawTextExW,NtSetInformationFile,CopyFileA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2566,ed5d70a13633a46355c0c2f9905ba29b7b74dfdb4db321...,1,NtDuplicateObject,RegCloseKey,LdrUnloadDll,NtSetInformationFile,RegQueryValueExA,NtTerminateProcess,NtQueryValueKey,RegQueryValueExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2567,ed5addbdbe5f56f108530148c71ab7db806ac9324395d0...,1,GetCursorPos,NtOpenSection,CoUninitialize,RegCloseKey,LdrUnloadDll,GetSystemInfo,RegQueryValueExA,NtTerminateProcess,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2568,ed4f4518e3120a4fd8ff6c61bf072d4de60264711a9196...,1,NtAllocateVirtualMemory,LdrGetProcedureAddress,SetUnhandledExceptionFilter,GetFileType,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetDllHandle,NtProtectVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
data["Unnamed: 176"].unique()

array([nan, 'WriteConsoleA'], dtype=object)

In [35]:
# Preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Codificar las llamadas a la API como números enteros
encoder = LabelEncoder()
data_encoded = data.apply(encoder.fit_transform)

# Dividir los datos en conjuntos de entrenamiento y prueba
X = data_encoded.drop('labels', axis=1)
y = data_encoded['labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [36]:
# Implementación del modelo
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=X_train.shape[1]),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(250, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)


Epoch 1/10
15/15 [==============================] - 1s 22ms/step - loss: 0.6713 - accuracy: 0.5853 - val_loss: 0.6217 - val_accuracy: 0.6654
Epoch 2/10
15/15 [==============================] - 0s 17ms/step - loss: 0.5399 - accuracy: 0.7782 - val_loss: 0.5111 - val_accuracy: 0.6965
Epoch 3/10
15/15 [==============================] - 0s 12ms/step - loss: 0.3858 - accuracy: 0.8299 - val_loss: 0.3678 - val_accuracy: 0.8366
Epoch 4/10
15/15 [==============================] - 0s 13ms/step - loss: 0.2716 - accuracy: 0.8955 - val_loss: 0.3118 - val_accuracy: 0.8755
Epoch 5/10
15/15 [==============================] - 0s 13ms/step - loss: 0.1947 - accuracy: 0.9350 - val_loss: 0.2957 - val_accuracy: 0.8651
Epoch 6/10
15/15 [==============================] - 0s 15ms/step - loss: 0.1222 - accuracy: 0.9705 - val_loss: 0.2473 - val_accuracy: 0.8911
Epoch 7/10
15/15 [==============================] - 0s 13ms/step - loss: 0.0699 - accuracy: 0.9878 - val_loss: 0.2357 - val_accuracy: 0.9053
Epoch 8/10
15

In [40]:
# Evaluación del modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

y_pred = model.predict(X_test)
y_pred_bin = np.round(y_pred).astype(int)

print(f'Accuracy: {accuracy_score(y_test, y_pred_bin)}')
print(f'Precision: {precision_score(y_test, y_pred_bin)}')
print(f'Recall: {recall_score(y_test, y_pred_bin)}')
print(f'ROC AUC: {roc_auc_score(y_test, y_pred_bin)}')


25/25 [==============================] - 0s 2ms/step
Accuracy: 0.8884565499351491
Precision: 0.8692493946731235
Recall: 0.9181585677749361
ROC AUC: 0.8880266523085206
